<h1><center> Frequent Itemset Mining </center></h1>

Para este exercício exploramos os dados de uma competição no Kaggle, disponível no link abaixo:

https://www.kaggle.com/c/instacart-market-basket-analysis

A empresa Instacart fornece serviços de compra e entrega através de um App. Entre seus objetivos estão identificar quais produtos os clientes:

- Comprariam novamente;
- Tentariam usar pela primeira vez;
- Deixariam para a próxima compra;

Usaremos a base transacional fornecida para buscar regras de associação. A idéia é buscar produtos que são frequentemente comprados em conjunto para orientar estratégias de cross-selling.

Para não implementar o algoritmo do zero, a biblioteca MLxtend será aplicada. Segue abaixo o link com detalhes da sua instalação no ambiente Anaconda;

https://anaconda.org/conda-forge/mlxtend

## Carregando os dados

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import gc

Carrega cestas de compra em um dataframe. Em função do volume de dados frequentemente usaremos estratégias para reduzir o uso de memória. Neste caso carregamos apenas as colunas indispensáveis: código da compra e do produto.

In [ ]:
url = r"https://raw.githubusercontent.com/brvnl/AplicacoesAprendizadoMaquina/main/order_products__train.csv"
df_raw = pd.read_csv(url, usecols=["order_id", "product_id"])

Checando o número de linhas, colunas e a memória ocupada:

In [ ]:
df_raw.info(memory_usage='deep')

Analisando uma amostra dos dados:

In [ ]:
df_raw.head(3)

Verificando carcaterísticas do dataset:

In [ ]:
print("Quantas compras diferentes? %d." %(len(df_raw["order_id"].unique())))
print("Quantos produtos diferentes? %d." %(len(df_raw["product_id"].unique())))

Carregando tabela com nomes dos produtos e convertendo em um dicionário:

In [ ]:
url_p = r"https://raw.githubusercontent.com/brvnl/AplicacoesAprendizadoMaquina/main/products.csv"
df_products_lookup = pd.read_csv(url_p)

d = dict(zip(df_products_lookup["product_id"], df_products_lookup["product_name"]))
del df_products_lookup

# Filtrando os dados

Por questão de uso de memória, pode ser necessário executar a análise por partes. Para isso, o código abaixo permite filtrar apenas compras/produtos de uma ilha ou departamento.

In [ ]:
def filter_aisle_department(df, aisle = None, department = None):
    url = r"https://github.com/brvnl/AplicacoesAprendizadoMaquina/blob/main/product_aisle_department_lookup.xlsx?raw=true"
    
    if aisle is not None:
        dft = pd.read_excel(url, sheet_name="Products", usecols=["product_id", "aisle"])
        dft = pd.merge(df, dft, left_on="product_id", right_on="product_id")
        dft = dft[dft["aisle"].isin(aisle)]
        dft.drop(columns="aisle", inplace=True)
        
    elif department is not None:
        dft = pd.read_excel(url, sheet_name="Products", usecols=["product_id", "department"])
        dft = pd.merge(df, dft, left_on="product_id", right_on="product_id")
        dft = dft[dft["department"].isin(department)]
        dft.drop(columns="department", inplace=True)
    
    else:
        dft = df
    
    print("Quantas compras diferentes? %d." %(len(dft["order_id"].unique())))
    print("Quantos produtos diferentes? %d." %(len(dft["product_id"].unique())))
    
    return dft

- Os parametros aisle e department devem ser sempre **None** ou na forma de lista (mesmo com um único elemento). 
- Na implementação atual o fitro ocorre ou por ilha ou por departamento, mas não junto. Adapte o código caso deseje o filtor duplo.

Segue abaixo alguns exemplos de uso:

In [ ]:
# Ex1.: Procurando relações entre produtos nos departamentos "beverages" e "snacks"
#df_raw = filter_aisle_department(df_raw, aisle = None, department = ["beverages", "snacks"])

# Ex2.: Procurando relações entre produtos na ilha de congelados
df_raw = filter_aisle_department(df_raw, aisle = ["frozen meals"], department = None)

## Ajustando o layout

Como estamos interessados em analisar características das compras individualmente, precisamos transformar os dados de forma que cada registro represente uma ordem. Para isso vamos distribuir o **order_id** nas linhas e **product_id** nas colunas com codificação One Hot Encoding. 

Poderiamos usar a coluna **add_to_cart_order** como valor, mas como só nos interessa se o produto foi comprado ou não, adicionamos uma nova coluna com a flag **1** nos produtos, que irá será usada para o One Hot Encoding.

In [ ]:
df_raw["comprado"] = 1

Abaixo executamos o pivot table para obter o novo layout. Nesta etapa o uso de memória cresce bastante, passando facilmente dos 16Gb de Ram. Para controlar este comportamento, ajuste a quantidade de linhas na função _head()_.

In [ ]:
# Para dados full podemos filtrar pela função head.
#df1 = df_raw.sample(400000).pivot_table(index='order_id', columns='product_id', values="comprado", fill_value=0)

# Caso os dados já estejam previamente filtrados podemos usar o dataframe diretamente
df1 = df_raw.pivot_table(index='order_id', columns='product_id', values="comprado", fill_value=0)

In [ ]:
df1.head()

Verificando quantidade de linhas, colunas e memória consumida pelo dataframe:

In [ ]:
df1.info(memory_usage='deep')

**_Caso o dataframe estaja ocupando muita memoria, execute o comando abaixo para desalocar RAM e tente uma quantidade menor de linhas no código de pivoteamento da tabela. Caso o uso de memória não diminua, reinicie o kernel do jupyter e execute o código desde o início._**

In [ ]:
del df1
gc.collect()

## Buscando regras com APRIORI

A biblioteca MLxtend trabalha com a mineração de regras em duas etapas:

- Na primeira são procurados conjuntos frequentes de acordo com um threshold para a métrica __suporte__.
- Na segunda etapa são avaliados dentre os conjuntos identificados, quais atendem um threshold mínimo de uma outra variável.

Mais detalhes sobre o uso da biblioteca estão disponíveis no link:
http://rasbt.github.io/mlxtend/api_subpackages/mlxtend.frequent_patterns/

In [ ]:
# Acha os conjuntos frequentes que atendem um suporte minimo (%)
frequent_itemsets = apriori(df1, min_support=0.05)

# Filtra os conjuntos pela métrica confiança
if len(frequent_itemsets):
    selected = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.02)
else:
    print("Não há itemsets que atendam ao mínimo suporte fornecido.")

Abaixo apresentamos os resultados, trocando os códigos dos produtos por sua descrição.

In [ ]:
def int_from_frozenset(row):
    return d[list(row)[0]]

if len(frequent_itemsets):
    selected["antecedents_name"] = selected["antecedents"].apply(int_from_frozenset)
    selected["consequents_name"] = selected["consequents"].apply(int_from_frozenset)
    selected.drop(columns=["antecedents", "consequents"])
else:
    print("Não há itemsets que atendam ao mínimo suporte fornecido.")

## Relações encontradas

Descreva no campo abaixo as relações mais interessantes encontradas e como você chegou a elas.

Sua resposta: